# Access EcoStress data from AppEEARS API
Note: /Users/morgansteckler/opt/miniconda3/envs/gee/lib/python3.8/site-packages/shapely/geos.py throws an error at line 113. I had to change the line to `free = CDLL(None).free` to prevent an error.

In [2]:
# Import packages 
import requests as r
import getpass, pprint, time, os, cgi, json
import geopandas as gpd 

In [3]:
# Set input directory, change working directory
inDir = '/Users/morgansteckler/Desktop/tropical_phenology_gee/morgan/appeears-api-getting-started-master@f64abca248a'
os.chdir(inDir)
api = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'

In [4]:
# Sign in
user = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')      # Input NASA Earthdata Login Username
password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')  # Input NASA Earthdata Login Password

Enter NASA Earthdata Login Username: ········
Enter NASA Earthdata Login Password: ········


In [5]:
# Get bearer token
token_response = r.post('{}login'.format(api), auth=(user, password)).json() # Insert API URL, call login service, provide credentials & return json
del user, password                                                           # Remove user and password information
token_response                                                               # Print response

{'token_type': 'Bearer',
 'token': 'omkV9ZFrqAbxQ2qAUIl8nl60maZJcHSd5ywgXN52tcAKvNN1FvhB1us37-nRcGLZs7KndRCqrx2iDqQFyWzBNg',
 'expiration': '2021-01-21T19:08:08Z'}

## 1. Look at all products and find ecostress

In [60]:
# List all products
product_response = r.get('{}product'.format(api)).json()
products = {p['ProductAndVersion']: p for p in product_response}
for key, val in products.items():
    name = val['ProductAndVersion']
    desc = val['Description']
    print(str(name) + ": " + str(desc))

GPW_DataQualityInd.004: Quality of Input Data for Population Count and Density Grids
GPW_UN_Adj_PopCount.004: UN-adjusted Population Count
GPW_UN_Adj_PopDensity.004: UN-adjusted Population Density
MCD12Q1.006: Land Cover Type
MCD12Q2.006: Land Cover Dynamics
MCD15A2H.006: Leaf Area Index (LAI) and Fraction of Photosynthetically Active Radiation (FPAR)
MCD15A3H.006: Leaf Area Index (LAI) and Fraction of Photosynthetically Active Radiation (FPAR)
MCD43A1.006: Bidirectional Reflectance Distribution Function (BRDF) and Albedo
MCD43A3.006: Bidirectional Reflectance Distribution Function (BRDF) and Albedo
MCD43A4.006: Bidirectional Reflectance Distribution Function (BRDF) and Albedo
MCD64A1.006: Burned Area (fire)
MOD09A1.006: Surface Reflectance Bands 1-7
MOD09GA.006: Surface Reflectance Bands 1-7
MOD09GQ.006: Surface Reflectance Bands 1-2
MOD09Q1.006: Surface Reflectance Bands 1-2
MOD10A1.005: Snow Cover
MOD10A1.006: Snow Cover (NDSI)
MOD10A2.005: Snow Cover
MOD10A2.006: Snow Cover
MOD11A1

In [61]:
# Get info on an Ecostress product
pprint.pprint(products['ECO4ESIPTJPL.001'])

{'Available': True,
 'DOI': '10.5067/ECOSTRESS/ECO4ESIPTJPL.001',
 'Deleted': False,
 'Description': 'Evaporative Stress Index PT-JPL',
 'DocLink': 'https://doi.org/10.5067/ECOSTRESS/ECO4ESIPTJPL.001',
 'Platform': 'ECOSTRESS',
 'Product': 'ECO4ESIPTJPL',
 'ProductAndVersion': 'ECO4ESIPTJPL.001',
 'RasterType': 'Swath',
 'Resolution': '70m',
 'Source': 'LP DAAC',
 'TemporalExtentEnd': 'Present',
 'TemporalExtentStart': '2018-07-09',
 'TemporalGranularity': 'ISS-dependent',
 'Version': '001'}


In [63]:
# Create a list of products I want
prods = ['ECO2LSTE.001'] # Lands surface temp and emissivity
prods.append('ECO3ETPTJPL.001') # Evapotranspiration
#prods.append('ECO4ESIPTJPL.001') # Evaporative stress index
prods

['ECO2LSTE.001', 'ECO3ETPTJPL.001']

## 2. Look at ecostress evapotranspiration product layers

In [64]:
# Request layers for the 2nd product (index 1) in the list: 'ECO3ETPTJPL.001'
canopy_response = r.get('{}product/{}'.format(api, prods[1])).json()
list(canopy_response.keys())

['EVAPOTRANSPIRATION_PT_JPL_ETcanopy',
 'EVAPOTRANSPIRATION_PT_JPL_ETdaily',
 'EVAPOTRANSPIRATION_PT_JPL_ETinst',
 'EVAPOTRANSPIRATION_PT_JPL_ETinstUncertainty',
 'EVAPOTRANSPIRATION_PT_JPL_ETinterception',
 'EVAPOTRANSPIRATION_PT_JPL_ETsoil']

In [65]:
# Show what info there is for a layer
canopy_response['EVAPOTRANSPIRATION_PT_JPL_ETcanopy']

{'AddOffset': '',
 'Available': True,
 'DataType': 'float32',
 'Description': 'Canopy Evaporation',
 'Dimensions': ['FakeDim0', 'FakeDim1'],
 'FillValue': '',
 'IsQA': False,
 'Layer': 'EVAPOTRANSPIRATION_PT_JPL_ETcanopy',
 'OrigDataType': 'float32',
 'OrigValidMax': 100.0,
 'OrigValidMin': 0.0,
 'QualityLayers': '',
 'QualityProductAndVersion': '',
 'ScaleFactor': '',
 'Units': '%',
 'ValidMax': 100.0,
 'ValidMin': 0.0,
 'XSize': 5400,
 'YSize': 5632}

In [66]:
# Create list of tuples with the layers I want from 'ECO3ETPTJPL.001'
layers = [(prods[1],'EVAPOTRANSPIRATION_PT_JPL_ETcanopy'),
          (prods[1],'EVAPOTRANSPIRATION_PT_JPL_ETdaily')]

## 3. Look at ecostress emissivity product layers

In [67]:
# Request layers for the 1st product (index 0) in the list: 'ECO2LSTE.001'
temp_response = r.get('{}product/{}'.format(api, prods[0])).json()  # Request layers for the 1st product (index 0) in the list: MCD15A3H.006
list(temp_response.keys())  

['SDS_Emis1',
 'SDS_Emis1_err',
 'SDS_Emis2',
 'SDS_Emis2_err',
 'SDS_Emis3',
 'SDS_Emis3_err',
 'SDS_Emis4',
 'SDS_Emis4_err',
 'SDS_Emis5',
 'SDS_Emis5_err',
 'SDS_EmisWB',
 'SDS_LST',
 'SDS_LST_err',
 'SDS_PWV',
 'SDS_QC']

In [68]:
temp_response['SDS_LST']

{'AddOffset': 0.0,
 'Available': True,
 'DataType': 'uint16',
 'Description': 'Land Surface Temperature',
 'Dimensions': ['FakeDim0', 'FakeDim1'],
 'FillValue': 0.0,
 'IsQA': False,
 'Layer': 'SDS_LST',
 'OrigDataType': 'uint16',
 'OrigValidMax': 65535,
 'OrigValidMin': 7500,
 'QualityLayers': "['SDS_QC']",
 'QualityProductAndVersion': 'ECO2LSTE.001',
 'ScaleFactor': 0.02,
 'Units': 'K',
 'ValidMax': 1310.7,
 'ValidMin': 150.0,
 'XSize': 5400,
 'YSize': 5632}

In [69]:
layers.append((prods[0],'SDS_LST'))

## 4. Look at evaporative stress index product layers

In [70]:
# Request layers for the 3rd product (index 2) in the list: 'ECO4ESIPTJPL.001'
#esi_response = r.get('{}product/{}'.format(api, prods[2])).json()  # Request layers for the 1st product (index 0) in the list: MCD15A3H.006
#list(esi_response.keys())

In [71]:
#esi_response['Evaporative_Stress_Index_PT_JPL_ESIavg']

In [72]:
#layers.append((prods[0], 'ESI'))

## 5. Submit a request
### 5a. Get a shapefile

In [73]:
# Create easy to process dictionary of products and layers I want
prodLayer = []
for l in layers:
    prodLayer.append({
            "layer": l[1],
            "product": l[0]
          })
prodLayer

[{'layer': 'EVAPOTRANSPIRATION_PT_JPL_ETcanopy', 'product': 'ECO3ETPTJPL.001'},
 {'layer': 'EVAPOTRANSPIRATION_PT_JPL_ETdaily', 'product': 'ECO3ETPTJPL.001'},
 {'layer': 'SDS_LST', 'product': 'ECO2LSTE.001'}]

In [74]:
# Save token for request
token = token_response['token']
head = {'Authorization': 'Bearer {}'.format(token)}

In [75]:
# Import shapefile
inDir = '/Users/morgansteckler/Desktop/tropical_phenology_gee/morgan/'
grid = gpd.read_file('{}/grid.shp'.format(inDir + os.sep)) # Read in shapefile as dataframe using geopandas
for col in grid.columns: # Print headers
    print(col)

geometry


In [76]:
# Select first plot
grid0 = grid[grid['geometry']=='0'].to_json() # Extract Grand Canyon NP and set to variable
grid0 = json.loads(grid0)

### 5b. Get a projection

In [77]:
# Show available projections
projections = r.get('{}spatial/proj'.format(api)).json()  # Call to spatial API, return projs as json
projections 

[{'Name': 'native',
  'Description': 'Native Projection',
  'Platforms': '',
  'Proj4': '',
  'Datum': '',
  'EPSG': '',
  'Units': '',
  'GridMapping': '',
  'Available': True},
 {'Name': 'geographic',
  'Description': 'Geographic',
  'Platforms': "['SRTM', 'ECOSTRESS', 'SSEBop ET', 'GPW', 'ASTER GDEM', 'NASADEM']",
  'Proj4': '+proj=longlat +datum=WGS84 +no_defs',
  'Datum': 'wgs84',
  'EPSG': 4326.0,
  'Units': 'degrees',
  'GridMapping': 'latitude_longitude',
  'Available': True},
 {'Name': 'sinu_modis',
  'Description': 'MODIS Sinusoidal',
  'Platforms': "['Combined MODIS', 'Terra MODIS', 'Aqua MODIS', 'S-NPP NASA VIIRS', 'Global WELD']",
  'Proj4': '+proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs',
  'Datum': '',
  'EPSG': '',
  'Units': 'meters',
  'GridMapping': 'sinusoidal',
  'Available': True},
 {'Name': 'albers_weld_alaska',
  'Description': 'WELD Albers Equal Area Alaska',
  'Platforms': "['WELD']",
  'Proj4': '+proj=aea +lat_1=55 +lat_2=6

In [78]:
projs = {}                                  # Create an empty dictionary
for p in projections: projs[p['Name']] = p  # Fill dictionary with `Name` as keys
list(projs.keys())                          # Print dictionary keys

['native',
 'geographic',
 'sinu_modis',
 'albers_weld_alaska',
 'albers_weld_conus',
 'albers_ard_alaska',
 'albers_ard_conus',
 'albers_ard_hawaii',
 'easegrid_2_global',
 'easegrid_2_north',
 'laea_sphere_19']

In [79]:
projs['geographic']

{'Name': 'geographic',
 'Description': 'Geographic',
 'Platforms': "['SRTM', 'ECOSTRESS', 'SSEBop ET', 'GPW', 'ASTER GDEM', 'NASADEM']",
 'Proj4': '+proj=longlat +datum=WGS84 +no_defs',
 'Datum': 'wgs84',
 'EPSG': 4326.0,
 'Units': 'degrees',
 'GridMapping': 'latitude_longitude',
 'Available': True}

### 5c. Combile JSON task object

In [80]:
task_name = input('Enter a Task Name: ')

Enter a Task Name: test1


In [89]:
task_type = ['area']        # Type of task, area or point
proj = projs['geographic']['Name']  # Set output projection 
outFormat = ['geotiff']  # Set output file format type
startDate = '07-01-2016'            # Start of the date range for which to extract data: MM-DD-YYYY
endDate = '07-31-2018'              # End of the date range for which to extract data: MM-DD-YYYY
recurring = False                   # Specify True for a recurring date range
#yearRange = [2000,2016]            # if recurring = True, set yearRange, change start/end date to MM-DD

In [90]:
task = {
    'task_type': task_type[0],
    'task_name': task_name,
    'params': {
         'dates': [
         {
             'startDate': startDate,
             'endDate': endDate
         }],
         'layers': prodLayer,
         'output': {
                 'format': {
                         'type': outFormat[0]}, 
                         'projection': proj},
         'geo': grid0,
    }
}

In [91]:
task_response = r.post('{}task'.format(api), json=task, headers=head).json()  # Post json to the API task service, return response as json
task_response  

{'task_id': 'b14c379b-f2d5-464d-8a4b-c4914dd96f4c', 'status': 'pending'}

In [92]:
task_id = task_response['task_id']
status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json()

starttime = time.time()
while r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'] != 'done':
    print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])
    time.sleep(20.0 - ((time.time() - starttime) % 20.0))
print(r.get('{}task/{}'.format(api, task_id), headers=head).json()['status'])

pending
done


In [93]:
inDir = inDir = '/Users/morgansteckler/Desktop/tropical_phenology_gee/morgan/'
destDir = os.path.join(inDir, task_name)                # Set up output directory using input directory and task name
if not os.path.exists(destDir):os.makedirs(destDir)     # Create the output directory

In [94]:
bundle = r.get('{}bundle/{}'.format(api,task_id)).json()  # Call API and return bundle contents for the task_id as json
bundle                                                    # Print bundle contents

{'files': [{'sha256': '6ed267d52577195e71e72f84f96306f4a0102fc38126a8793f082ef9d8dbd33f',
   'file_id': 'ae581515-7e32-4d9b-bb46-aebe562f3177',
   'file_name': 'test1-request.json',
   'file_size': 913,
   'file_type': 'json'},
  {'sha256': 'c8dadfa39ddd90669a58351e3b354f0715c83c4d6c4aef662a827a802bb9dcf8',
   'file_id': '10f42af6-e6ef-47d3-a2bd-f055c2d0dfbb',
   'file_name': 'test1-ECO2LSTE-001-metadata.xml',
   'file_size': 20232,
   'file_type': 'xml'},
  {'sha256': '888391c1d4903446b4719332bc3e3438128ae8cb858dffdcc071e20849468a55',
   'file_id': '4179a467-448f-4b2b-9ddc-a0756e5d7c55',
   'file_name': 'test1-ECO3ETPTJPL-001-metadata.xml',
   'file_size': 20213,
   'file_type': 'xml'},
  {'sha256': 'e4b8b95891b59b88f7607dc0a89420335a0ecceb80318171125816b4d70f8089',
   'file_id': '16423e23-384e-4969-b683-130e32627690',
   'file_name': 'README.md',
   'file_size': 22748,
   'file_type': 'txt'}],
 'created': '2021-01-19T21:45:14.196496',
 'task_id': 'b14c379b-f2d5-464d-8a4b-c4914dd96f4c

In [95]:
files = {}                                                       # Create empty dictionary
for f in bundle['files']: files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values
files    

{'ae581515-7e32-4d9b-bb46-aebe562f3177': 'test1-request.json',
 '10f42af6-e6ef-47d3-a2bd-f055c2d0dfbb': 'test1-ECO2LSTE-001-metadata.xml',
 '4179a467-448f-4b2b-9ddc-a0756e5d7c55': 'test1-ECO3ETPTJPL-001-metadata.xml',
 '16423e23-384e-4969-b683-130e32627690': 'README.md'}

In [88]:
for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), stream=True)                                # Get a stream to the bundle file
    filename = os.path.basename(cgi.parse_header(dl.headers['Content-Disposition'])[1]['filename'])  # Parse the name from Content-Disposition header 
    filepath = os.path.join(destDir, filename)                                                       # Create output file path
    with open(filepath, 'wb') as f:                                                                  # Write file to dest dir
        for data in dl.iter_content(chunk_size=8192): f.write(data) 
print('Downloaded files can be found at: {}'.format(destDir))

Downloaded files can be found at: /Users/morgansteckler/Desktop/tropical_phenology_gee/morgan/test1
